#**Домашнее задание №4: fine-tuning**

# Цель задания

Наша задача заключается в выполнении fine-tuning для модели с использованием технологии LoRa на основе данных из ДЗ-2: так, будем использовать модель **RuadaptQwen2.5-1.5B-instruct**, показавшую более высокие результаты, и датасет **ai-forever/MERA** (сабсет rcb). Сравним качество модели до и после дообучения на тестовой выборке (т.е. split *'validation'* из рассматриваемого датасета).

Рассмотрим содержимое датасета *ai-forever/MERA* (сабсет *rcb*):
*   instruction: содержит задачу-инструкцию (анализ гипотезы на основе некоторого контекста),
*   inputs: тексты с контекстом и гипотезой для поля instructions,
*   outputs: метка в виде цифры (1, 2 или 3), означающая правильный ответ,
*   meta: доп. информация.

Поля 'instruction' и 'inputs' объединяем в единый промпт, после чего модель должна сгенерировать цифру-ответ, и, если ответ правильный, он должен совпасть с соответствующей цифрой в поле 'outputs'.

#План работы

Для текущего задания нам потребуется:
*   Разбить данные на train и test;
*   Взять и подготовить модель RuadaptQwen2.5-1.5B-instruct;
*   Проанализировать качество исходной модели (дополнительно рассмотрим поведение на few-shot промпте);
*   Провести fine-tuning с использованием LoRa, используя split 'train';
*   Сравнить качество (accuracy) до и после fine-tuning, сделать выводы.

###Подготовка

Начинаем с импортов и загрузки:

In [ ]:
!pip install accelerate       # для ускорения обучения
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install peft

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForTokenClassification, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftModel
from torch.utils.data import Dataset
from tqdm import tqdm
from typing import List, Dict

In [ ]:
dataset = load_dataset("ai-forever/MERA", "rcb")
train_dataset = dataset["train"]
test_dataset = dataset["validation"]

print(f"\nРазмер Train: {len(train_dataset)}, Размер Test (точнее, split 'Validation'): {len(test_dataset)}")
for i in range(5):
    print(f"Sample {i}: outputs = '{test_dataset[i]['outputs']}'")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Размер Train: 438, Размер Test (точнее, split 'Validation'): 220
Sample 0: outputs = '3'
Sample 1: outputs = '2'
Sample 2: outputs = '2'
Sample 3: outputs = '3'
Sample 4: outputs = '1'


Напишем функцию *create_messages*, которая формирует формат сообщений:

* system для роли модели;
* user для роли пользователя;
* assistant для хранения правильного ответа, используется при обучении.

Для каждой записи в обучающем и тестовом датасете будем формировать список сообщений в нужном формате:

In [ ]:
def create_messages(sample):
    return [
        {"role": "system", "content": "Вы - самый умный и логичный в мире помощник. Пользователь будет предоставлять, во-первых, текст с описанием ситуации, во-вторых, гипотезу. Тебе нужно понять, есть ли взаимосвязь гипотезы с описанной ситуацией, и выбрать один из трех вариантов: 1 - гипотеза следует из ситуации, 2 - гипотеза противоречит ситуации, 3 - гипотеза независима от ситуации."},
        {"role": "user", "content": f"{sample['instruction']}\n{sample['inputs']}"},
        {"role": "assistant", "content": str(sample['outputs'])}
    ]

train_messages = [create_messages(sample) for sample in train_dataset]
test_messages = [create_messages(sample) for sample in test_dataset]

Загрузим нашу модель и соответствующий ей токенизатор:

In [ ]:
model_name = "RefalMachine/RuadaptQwen2.5-1.5B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype=torch.float16,
    attn_implementation="sdpa"
)

Далее мы создадим класс ChatDataset для подготовки данных к обучению.

* метод **__init__**: для каждой записи с помощью метода convert_record создаются тензоры с входными данными (input_ids), метками (labels) и attention_mask;

* метод **convert_record** обрабатывает каждое сообщение из диалога, преобразуя его в последовательность id токенов с помощью apply_chat_template токенизатора.

Полученные тензоры затем используются для формирования train_chat_dataset и test_chat_dataset.

In [ ]:
class ChatDataset(Dataset):
    def __init__(
        self,
        original_records: List[Dict],
        tokenizer: AutoTokenizer,
        max_tokens_count: int,
        only_target_loss: bool = True,
        labels_pad_token_id: int = -100,
    ):
        self.original_records = original_records
        self.tokenizer = tokenizer
        self.max_tokens_count = max_tokens_count
        self.only_target_loss = only_target_loss
        self.labels_pad_token_id = labels_pad_token_id
        self.records = []
        for record in tqdm(original_records):
            tensors = self.convert_record(record)
            if tensors is None:
                continue
            self.records.append(tensors)

    def __len__(self):
        return len(self.records)

    def __getitem__(self, index):
        return self.records[index]

    def convert_record(self, record):
        input_ids, labels = [], []
        for message in record:
            message_input_ids = self.tokenizer.apply_chat_template(
                [message],
                add_special_tokens=False,
                tokenize=True,
                add_generation_prompt=False,
            )
            if message_input_ids[0] == self.tokenizer.bos_token_id:
                message_input_ids = message_input_ids[1:]
            message_labels = message_input_ids.copy()
            if len(input_ids) + len(message_input_ids) > self.max_tokens_count - 2:
                break
            if message["role"] != "assistant" and self.only_target_loss:
                message_labels = [self.labels_pad_token_id] * len(message_input_ids)
            input_ids.extend(message_input_ids)
            labels.extend(message_labels)
        if not input_ids:
            return None
        if input_ids[0] != self.tokenizer.bos_token_id:
            input_ids.insert(0, self.tokenizer.bos_token_id)
            labels.insert(0, self.labels_pad_token_id)
        if input_ids[-1] != self.tokenizer.eos_token_id:
            input_ids.append(self.tokenizer.eos_token_id)
            labels.append(self.tokenizer.eos_token_id)
        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(labels)
        attention_mask = input_ids.new_ones(input_ids.size())
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }

max_tokens_count = 1024
train_chat_dataset = ChatDataset(train_messages, tokenizer, max_tokens_count)
test_chat_dataset = ChatDataset(test_messages, tokenizer, max_tokens_count)

100%|██████████| 220/220 [00:00<00:00, 317.81it/s]


Напишем функцию *generate*: она принимает сообщения, модель, токенизатор и конфигурацию генерации. Сначала сообщения преобразуются в input_ids с помощью apply_chat_template, затем выполняется генерация. Далее из полученных токенов удаляются входные и результат декодируется в текст:

In [ ]:
def generate(messages, model, tokenizer, generation_config):
    input_ids = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_special_tokens=False,
        add_generation_prompt=True,
    ).to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            generation_config=generation_config,
        )
    output_ids = output_ids[:, input_ids.shape[1]:]
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output.strip()

generation_config = GenerationConfig(
    max_new_tokens=1,  # ибо генерим только одну цифру
    do_sample=True,
    temperature=0.01,
    top_k=3,
    top_p=0.7,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

###Оценка модели до fine-tuning

In [ ]:
correct = 0
total = 0

for sample in test_dataset:
    user_message = {"role": "user", "content": f"{sample['instruction']}\n{sample['inputs']}"}
    true_answer = str(sample['outputs'])

    messages = [
        {"role": "system", "content": "Вы - самый умный и логичный в мире помощник. Пользователь будет предоставлять, во-первых, текст с описанием ситуации, во-вторых, гипотезу. Тебе нужно понять, есть ли взаимосвязь гипотезы с описанной ситуацией, и выбрать один из трех вариантов: 1 - гипотеза следует из ситуации, 2 - гипотеза противоречит ситуации, 3 - гипотеза независима от ситуации."},
        user_message
    ]

    generated_answer = generate(messages, model, tokenizer, generation_config).strip()
    print(f"True: '{true_answer}', Generated: '{generated_answer}'")

    if generated_answer == true_answer:
        correct += 1
    total += 1


accuracy = correct / total
print(f"Accuracy: {accuracy}")

True: '3', Generated: '1'
True: '2', Generated: '1'
True: '2', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '1'
True: '3', Generated: '1'
True: '2', Generated: '1'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '3', Generated: '1'
True: '2', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '1'
True: '2', Generated: '1'
True: '2', Generated: '1'
True: '3', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '1'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '3', Generated: '1'
True: '2', Generated: '1'
True: '1', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '1'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '3', Generated: '1'
True: '2', Generated: '1'
True: '1', G

Как видим, пока что качество очень низкое и находится на уровне случайного угадывания.

###Оценка модели до fine-tuning + few-shot

Рассмотрим поведение модели при добавлении в промпт нескольких примеров, по одному для каждого класса ответа:

In [ ]:
# выбираем несколько примеров из train_dataset (первые три - разных классов)
few_shot_examples = [
    {"instruction": train_dataset[0]['instruction'], "inputs": train_dataset[0]['inputs'], "outputs": train_dataset[0]['outputs']},
    {"instruction": train_dataset[1]['instruction'], "inputs": train_dataset[1]['inputs'], "outputs": train_dataset[1]['outputs']},
    {"instruction": train_dataset[2]['instruction'], "inputs": train_dataset[2]['inputs'], "outputs": train_dataset[2]['outputs']}
]


def create_few_shot_messages(sample, few_shot_examples):
    messages = [
        {"role": "system", "content": "Вы - самый умный и логичный в мире помощник. Пользователь будет предоставлять, во-первых, текст с описанием ситуации, во-вторых, гипотезу. Тебе нужно понять, есть ли взаимосвязь гипотезы с описанной ситуацией, и выбрать один из трех вариантов: 1 - гипотеза следует из ситуации, 2 - гипотеза противоречит ситуации, 3 - гипотеза независима от ситуации."},
    ]

    for example in few_shot_examples:
        messages.append({"role": "user", "content": f"{example['instruction']}\n{example['inputs']}"} )
        messages.append({"role": "assistant", "content": example['outputs']})

    messages.append({"role": "user", "content": f"{sample['instruction']}\n{sample['inputs']}"})
    return messages

In [ ]:
correct = 0
total = 0

for sample in test_dataset:
    messages = create_few_shot_messages(sample, few_shot_examples)
    true_answer = str(sample['outputs'])  # Истинный ответ

    generated_answer = generate(messages, model, tokenizer, generation_config).strip()
    print(f"True: '{true_answer}', Generated: '{generated_answer}'")

    if generated_answer == true_answer:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Accuracy (+Few-shot learning): {accuracy}")

True: '3', Generated: '1'
True: '2', Generated: '1'
True: '2', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '1'
True: '3', Generated: '1'
True: '2', Generated: '2'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '3', Generated: '1'
True: '2', Generated: '2'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '1'
True: '2', Generated: '1'
True: '2', Generated: '1'
True: '3', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '2'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '1', Generated: '1'
True: '1', Generated: '1'
True: '3', Generated: '2'
True: '1', Generated: '1'
True: '1', Generated: '1'
True: '3', Generated: '2'
True: '3', Generated: '1'
True: '2', Generated: '1'
True: '1', Generated: '1'
True: '1', Generated: '1'
True: '2', Generated: '3'
True: '1', Generated: '1'
True: '3', Generated: '1'
True: '3', Generated: '1'
True: '2', Generated: '2'
True: '1', G

Как можно видеть, качество немного улучшилось. Но что будет, если мы специально дообучим нашу модель для данной задачи?

###Реализация fine-tuning

Настроим конфигурацию для Lora:

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

model = get_peft_model(model, lora_config)

Настроим параметры обучения (текущие параметры найдены экспериментальным путем для более лучшего результата). Справа от каждого параметра - пояснение (найденное), для чего он нужен:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned_model",        # Папка для сохранения модели
    overwrite_output_dir=True,             # Перезаписывать папку, если она существует
    per_device_train_batch_size=1,         # Размер батча на устройство
    gradient_accumulation_steps=8,         # Накопление градиентов для имитации батча размером n
    learning_rate=7e-5,                    # Скорость обучения
    num_train_epochs=3,                    # Количество эпох
    evaluation_strategy="epoch",           # Оценка после каждой эпохи
    save_strategy="epoch",                 # Сохранение после каждой эпохи
    logging_steps=10,                      # Логирование каждые n шагов
    fp16=True,                             # Использование mixed precision для ускорения
    seed=42,                               # Фиксация случайности
    report_to="none"                       # Отключение отчетов (например, в wandb)
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Инициализацируем Trainer, запускаем обучение (около 4-5 минут):

In [ ]:
from transformers import Trainer, DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_chat_dataset,
    eval_dataset=test_chat_dataset,
    data_collator=data_collator,
)

trainer.train()

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,0.158400,No log
2,0.124300,No log


TrainOutput(global_step=162, training_loss=0.30456860694620347, metrics={'train_runtime': 296.5698, 'train_samples_per_second': 4.431, 'train_steps_per_second': 0.546, 'total_flos': 2347273669410816.0, 'train_loss': 0.30456860694620347, 'epoch': 2.949771689497717})

Сохранение и загрузка дообученной модели:

In [ ]:
model.save_pretrained("./finetuned_model_lora")
tokenizer.save_pretrained("./finetuned_model_lora")

base_model = AutoModelForCausalLM.from_pretrained("RefalMachine/RuadaptQwen2.5-1.5B-instruct",
                                                  device_map="cuda",
                                                  torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base_model, "./finetuned_model_lora")
model = model.merge_and_unload()    #  адаптеры + "базовая" модель

###Оценка модели после fine-tuning

In [ ]:
correct = 0
total = 0

for sample in test_dataset:
    user_message = {"role": "user", "content": f"{sample['instruction']}\n{sample['inputs']}"}
    true_answer = str(sample['outputs'])

    messages = [
        {"role": "system", "content": "Вы - самый умный и логичный в мире помощник. Пользователь будет предоставлять, во-первых, текст с описанием ситуации, во-вторых, гипотезу. Тебе нужно понять, есть ли взаимосвязь гипотезы с описанной ситуацией, и выбрать один из трех вариантов: 1 - гипотеза следует из ситуации, 2 - гипотеза противоречит ситуации, 3 - гипотеза независима от ситуации. За каждый правильный ответ ты будешь получать ТРИЛЛИОН долларов чаевых."},
        user_message
    ]

    generated_answer = generate(messages, model, tokenizer, generation_config).strip()
    if generated_answer == true_answer:
        correct += 1
    total += 1


accuracy_after = correct / total
print(f"Accuracy после fine-tuning: {accuracy_after}")

Accuracy после fine-tuning: 0.6


# Выводы

Итак, в данном задании мы дообучили инструктивную модель с использованием LoRa. Сначала данные были подготовлены: датасет был разбит на обучающую и тестовую выборки, а текстовые данные преобразованы в диалоговый формат. Далее были загружены базовая модель и токенизатор, а также реализован класс для подготовки данных. Эксперимент с few-shot показал, что добавление нескольких примеров в промпт уже может заметно улучшить точность модели (+0.07). После применения LoRa-адаптеров и настройки параметров обучения качество модели существенно выросло. При самом удачном запуске дообучения удалось добиться результата в **0.64**; в целом же, итоговый accuracy варьируется в районе значения **0.6**, и точность ответов с fine-tuning почти в 2 раза выше по сравнению с тем результатом, что был получен при "базовой" модели.

Цель задания достигнута!








